In [1]:
!pip install transformers peft bitsandbytes datasets trl accelerate torch flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 37.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 32.6 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [2]:
import torch
import os
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig # No need for get_peft_model explicitly with SFTTrainer
from trl import SFTTrainer

In [3]:
# --- 1. Configuration ---

base_model_name = "microsoft/phi-3-mini-4k-instruct"
dataset_path = "/content/drive/MyDrive/train_json_extraction.jsonl"

# Ensure Drive is mounted first:
# from google.colab import drive
# drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/my-phi3-json-extraction-adapter"
print(f"JSON extraction adapter will be saved to: {output_dir}")

JSON extraction adapter will be saved to: /content/drive/MyDrive/my-phi3-json-extraction-adapter


In [4]:
# --- 2. Load Model, Tokenizer, and Quantization (QLoRA) ---
print("Loading base model and tokenizer (4-bit)...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    # attn_implementation="flash_attention_2" # Optional: Use Flash Attention if available/compatible for potential speedup
)
model.config.use_cache = False # Disable cache for training
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
# Important: Set padding side based on SFTTrainer/model expectations. 'right' is common.
tokenizer.padding_side = "right"

Loading base model and tokenizer (4-bit)...


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [5]:
# --- 3. Configure LoRA ---
print("Configuring LoRA...")
peft_config = LoraConfig(
    r=16,                # Rank of LoRA matrices (16 or 32 is common)
    lora_alpha=32,       # Alpha scaling factor (often 2*r)
    lora_dropout=0.05,   # Dropout probability for LoRA layers
    bias="none",
    task_type="CAUSAL_LM",
    # Target modules can vary slightly per model. These are common for Phi-3.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

Configuring LoRA...


In [6]:
# --- 4. Load Your Formatted Training Data ---
print(f"Loading dataset from: {dataset_path}")
try:
    # Ensure it loads correctly, verify path if error occurs
    dataset = load_dataset("json", data_files=dataset_path, split="train")
    print(f"Dataset loaded with {len(dataset)} examples.")
    # You can uncomment below to double-check the first formatted example
    # print("First example format check:\n", dataset[0]['text'][:500] + "...") # Print first 500 chars
except Exception as e:
    print(f"Error loading dataset: {e}")
    print(f"Please ensure '{dataset_path}' exists and is correctly formatted.")
    raise

Loading dataset from: /content/drive/MyDrive/train_json_extraction.jsonl


Generating train split: 0 examples [00:00, ? examples/s]

Dataset loaded with 400 examples.


In [7]:
# --- 5. Set Up Training Arguments ---
print("Setting up training arguments...")
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,       # Start with 1 due to potentially long sequences (schema+text)
    gradient_accumulation_steps=8,       # Increase accumulation to simulate larger batch size (1*8=8)
    # num_train_epochs=3,                  # Train for a few epochs (adjust based on results)
    num_train_epochs=1,                  # Train for a few epochs (adjust based on results)
    learning_rate=2e-4,                  # Standard learning rate for QLoRA
    logging_steps=10,                    # Log more frequently for smaller datasets
    optim="paged_adamw_8bit",
    save_strategy="epoch",               # Save at the end of each epoch
    # save_steps=50,                     # Or save every N steps if preferred
    # evaluation_strategy="no",
    # push_to_hub=output_dir.startswith("YourUsername/"),
    fp16=False,
    bf16=torch.cuda.is_bf16_supported(), # Use bfloat16 if available
    max_steps=-1,                        # Train for num_train_epochs unless max_steps is set
    warmup_ratio=0.03,
    group_by_length=True,                # Group sequences of similar length for efficiency
    lr_scheduler_type="constant",
    report_to="tensorboard",
    gradient_checkpointing=True,         # Crucial for saving memory
    gradient_checkpointing_kwargs={'use_reentrant':False},
)

Setting up training arguments...


In [8]:
# --- 6. Initialize the SFTTrainer ---
print("Initializing SFTTrainer...")
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    # dataset_text_field="text",
    # max_seq_length=1024,                 # Adjust based on average length in your data vs GPU memory. May need smaller (e.g., 512) or larger (e.g., 2048)
    # tokenizer=tokenizer,
    # args=training_args,
    # packing=False,                       # Set packing=False for instruction tuning datasets
)

Initializing SFTTrainer...


Adding EOS to train dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (9057 > 4096). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

In [9]:
# --- 7. Start Fine-Tuning ---
print("\nStarting fine-tuning for JSON extraction...")
train_result = trainer.train()
print("Fine-tuning finished.")


Starting fine-tuning for JSON extraction...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jeffpachecoo (jeffpachecoo-unisatc) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,0.596600


KeyboardInterrupt: 

In [ ]:
# --- 8. Save the Final Adapter ---
print(f"Saving the final JSON extraction adapter model to {output_dir}")
trainer.save_model(output_dir) # Saves only the adapter

In [ ]:
# Optional: Log and save metrics
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

print("\nJSON extraction adapter saved successfully!")
print(f"The 'adapter_path' for your next inference script is: '{output_dir}'")